In [ ]:

#Using genetic Algo to solve and find the best moves in connect four game:

import random
class GeneticConnectFour:

    def __init__(self, population_size, mutation_rate, crossover_rate):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

        self.population = []
        for _ in range(population_size):
            self.population.append(self.generate_random_individual())

    def generate_random_individual(self):
        individual = []
        for _ in range(7):
            individual.append(random.randint(0, 6))
        return individual

    def fitness(self, individual):
      board = [[None for _ in range(7)] for _ in range(6)]
      for row in range(6):
          for col in range(7):
              if col == individual[row]:
                  board[row][col] = 1
              else:
                  board[row][col] = 0

      for row in range(6):
          for col in range(4):
              if (
                  board[row][col] == board[row][col + 1] == board[row][col + 2] == board[row][col + 3] == 1
                  or (row + 3 < 6 and board[row][col] == board[row + 1][col + 1] == board[row + 2][col + 2] == board[row + 3][col + 3] == 1)
                  or (row + 3 < 6 and board[row][col] == board[row + 1][col] == board[row + 2][col] == board[row + 3][col] == 1)
                  or (row + 3 < 6 and col - 3 >= 0 and board[row][col] == board[row + 1][col - 1] == board[row + 2][col - 2] == board[row + 3][col - 3] == 1)
              ):
                  return 1

      return 0


    def evolve(self):
        new_population = []

        # Select the best individuals.
        best_individuals = sorted(self.population, key=self.fitness, reverse=True)[:self.population_size // 2]

        # Mutate the best individuals.
        for individual in best_individuals:
            if random.random() < self.mutation_rate:
                i = random.randint(0, 6)
                j = random.randint(0, 6)
                print(f"Mutation occurred: Changed individual[{i}] to {j}.")
                individual[i] = j

        # Cross the best individuals.
        for i in range(0, len(best_individuals) - 1, 2):
            if random.random() < self.crossover_rate:
                parent1 = best_individuals[i]
                parent2 = best_individuals[i+1]
                child1 = parent1[:]
                child2 = parent2[:]

                for i in range(7):
                    j = random.randint(0, 6)
                    child1[i] = parent2[i]
                    child2[i] = parent1[i]

                print("Crossover occurred:")
                print(f"Parent 1      : {parent1}")
                print(f"Parent 2      : {parent2}")
                print(f"Child 1       : {child1}")
                print(f"Child 2       : {child2}")
                print()

                new_population.append(child1)
                new_population.append(child2)

        # Fill the remaining population with random individuals
        while len(new_population) < self.population_size:
            new_population.append(self.generate_random_individual())

        # Replace the old population with the new population.
        self.population = new_population

    def run(self, num_generations):
        for generation in range(num_generations):
            print(f"Generation {generation+1}/{num_generations}")
            self.evolve()

            # Calculate and display the fitness score for the best individual in the current generation.
            best_individual = sorted(self.population, key=self.fitness, reverse=True)[0]
            fitness_score = self.fitness(best_individual)
            print(f"Fitness Score: {fitness_score}\n")

        best_individual = sorted(self.population, key=self.fitness, reverse=True)[0]

        print("\nEvolution complete.")
        print(f"The best individual after {num_generations} generations has a fitness of {self.fitness(best_individual)}.\n")
        print("Board representation:")
        print("----------------------")
        for row in range(7):
            for col in range(7):
                if col == best_individual[row]:
                    print("1 ", end="")
                else:
                    print("0 ", end="")
            print()
        print("----------------------")
        print(f"\nBest individual: {best_individual}")

        # Make the best individual win the game.
        for i in range(7):
            for j in range(7):
                if j == best_individual[i]:
                    print("1 ", end="")
                else:
                    print("0 ", end="")
            print()

if __name__ == "__main__":
    population_size = 100
    mutation_rate = 0.01
    crossover_rate = 0.05
    num_generations = 1000

    game = GeneticConnectFour(population_size, mutation_rate, crossover_rate)
    game.run(num_generations)


In [ ]:
import random

a = (random.randint(0,10))
print(a)

9


In [ ]:
import random
import math

class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.wins = 0
        self.visits = 0

    def add_child(self, child_state):
        child = Node(child_state, self)
        self.children.append(child)

    def update(self, result):
        self.wins += result
        self.visits += 1

    def select_child(self, exploration_param):
        best_child = None
        best_score = float("-inf")
        for child in self.children:
            score = child.wins / child.visits + exploration_param * math.sqrt(
                2 * math.log(self.visits) / child.visits
            )
            if score > best_score:
                best_score = score
                best_child = child
        return best_child

def simulate_random_game(state):
    current_state = state
    while not current_state.is_terminal():
        move = random.choice(current_state.get_valid_moves())
        current_state = current_state.get_next_state(move)
    winner = current_state.get_winner()
    if winner == 'X':
        return 1
    elif winner == 'O':
        return 0
    else:
        return 0.5

def run_mcts(state, num_simulations):
    root = Node(state)

    for _ in range(num_simulations):
        node = root
        while node.children:
            node = node.select_child(1.4)  # Exploration parameter (adjustable)

        if node.visits > 0:
            state = node.state.get_random_child_state()
        else:
            state = node.state

        result = simulate_random_game(state)
        while node:
            node.update(result)
            result = 1 - result  # Switch the result for the parent node
            node = node.parent

    if root.children:
        best_child = max(root.children, key=lambda child: child.visits)
        return best_child.state.get_last_move()
    else:
        # If no simulations were run, return a default move (e.g., the first valid move)
        return state.get_valid_moves()[0]


# Rest of the code remains the same



class ConnectFourState:
    def __init__(self):
        self.board = [[' ' for _ in range(7)] for _ in range(6)]
        self.current_player = 'X'

    def is_terminal(self):
        # Check if the game has been won or there are no more valid moves
        return self.get_winner() or not self.get_valid_moves()

    def get_valid_moves(self):
        # Return a list of valid moves (column numbers)
        valid_moves = []
        for col in range(7):
            if self.board[0][col] == ' ':
                valid_moves.append(col)
        return valid_moves

    def get_next_state(self, move):
        # Return a new state after applying the move
        next_state = ConnectFourState()
        next_state.board = [row[:] for row in self.board]  # Copy the current board
        for row in range(5, -1, -1):
            if next_state.board[row][move] == ' ':
                next_state.board[row][move] = self.current_player
                break
        next_state.current_player = 'O' if self.current_player == 'X' else 'X'
        return next_state

    def get_winner(self):
        # Return the winning player ('X' or 'O') if there is a winner, else None
        # Check rows
        for row in range(6):
            for col in range(4):
                if (
                    self.board[row][col] == self.board[row][col + 1] == self.board[row][col + 2] == self.board[row][col + 3] != ' '
                ):
                    return self.board[row][col]

        # Check columns
        for col in range(7):
            for row in range(3):
                if (
                    self.board[row][col] == self.board[row + 1][col] == self.board[row + 2][col] == self.board[row + 3][col] != ' '
                ):
                    return self.board[row][col]

        # Check diagonal \
        for row in range(3):
            for col in range(4):
                if (
                    self.board[row][col] == self.board[row + 1][col + 1] == self.board[row + 2][col + 2] == self.board[row + 3][col + 3] != ' '
                ):
                    return self.board[row][col]

        # Check diagonal /
        for row in range(3):
            for col in range(3, 7):
                if (
                    self.board[row][col] == self.board[row + 1][col - 1] == self.board[row + 2][col - 2] == self.board[row + 3][col - 3] != ' '
                ):
                    return self.board[row][col]

        return None

    def get_random_child_state(self):
        # Return a random child state (random valid move applied)
        valid_moves = self.get_valid_moves()
        random_move = random.choice(valid_moves)
        return self.get_next_state(random_move)

    def get_last_move(self):
        # Return the last move played
        for col in range(7):
            for row in range(5, -1, -1):
                if self.board[row][col] != ' ':
                    return col

# Create an initial game state
initial_state = ConnectFourState()

# Run MCTS to get the best move
best_move = run_mcts(initial_state, num_simulations=1000)

print("Best Move:", best_move)


Best Move: 0
